In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd

import csv
import codecs

In [2]:
train_data = pd.read_csv(r'train.csv')

In [3]:
train_data

,sentence,relation,head,head_type,head_offset,tail,tail_type,tail_offset
0,《逐风行》是百度文学旗下纵横中文网签约作家清水秋风创作的一部东方玄幻小说，小说已于2014-...,连载网站,逐风行,网络小说,1,纵横中文网,网站,12
1,谢万松，字树人，湖北省武汉市人，武汉钢铁集团公司联合焦化公司退体职工，生于1940年,出生地,谢万松,人物,0,湖北省武汉市,地点,8
2,《娘家的故事第二部》是张玲执导，林在培、何赛飞等主演的电视剧,导演,娘家的故事第二部,影视作品,1,张玲,人物,11
3,九玄珠是在纵横中文网连载的一部小说，作者是龙马,连载网站,九玄珠,网络小说,0,纵横中文网,网站,5
4,《下半生留住你一直相爱》是商人醉猫写的网络小说连载于17k小说网,连载网站,下半生留住你一直相爱,网络小说,1,17k小说网,网站,26
...,...,...,...,...,...,...,...,...
3995,演说上中学时，虽然没出过县城没坐过汽车，但我已从课本上知道英国首都伦敦有个海德公园，那里有个...,所在城市,海德公园,景点,37,伦敦,城市,33
3996,项目建设借鉴伦敦皇家园林——海德公园的营建思路，将城市生活与自然生态无缝对接，创造出优雅、国...,所在城市,海德公园,景点,14,伦敦,城市,6
3997,梁启超故居，有多处，分别位于北京、天津、新会等地,所在城市,梁启超故居,景点,0,天津,城市,17
3998,2000年，上海城隍庙修复一期工程基本结束，并于同年11月14日举行住持升座仪式，上海市道教...,所在城市,城隍庙,景点,8,上海市,城市,41


In [4]:
test_data = pd.read_csv(r'test.csv')
test_data

,sentence,relation,head,head_type,head_offset,tail,tail_type,tail_offset
0,孔正锡，导演，2005年以一部温馨的爱情电影《长腿叔叔》敲开电影界大门,导演,长腿叔叔,影视作品,23,孔正锡,人物,0
1,《我家有喜2》（又名《开门见喜》）是由丁仰国导演执导，张佳编剧，海陆、高梓淇、万茜、李佳航、...,导演,我家有喜,影视作品,1,丁仰国,人物,19
2,解福印导演还表示这次虽然选用新人做主演，但对他们的表现绝对有信心，“跟以往的功夫片不同，《功...,导演,功夫世家,影视作品,45,解福印,人物,0
3,《疯狂艳唇》是佐佐木浩久执导的一部恐怖剧情电影，本电影由新人三轮瞳映画初主演,导演,疯狂艳唇,影视作品,1,佐佐木浩久,人物,7
4,步入二十一世纪，天愿大介似乎又对导筒发生了兴趣，陆续推出了几部电影，《世界上最美丽的夜晚》是...,导演,世界上最美丽的夜晚,影视作品,35,天愿大介,人物,8
...,...,...,...,...,...,...,...,...
995,建国后，南京市人民政府于1951年将莫愁湖列为“第一区人民公园”,所在城市,莫愁湖,景点,18,南京,城市,4
996,重庆中央公园是继纽约·中央公园、伦敦·海德公园之后的世界第三大城市中央公园，也是亚洲最大的城...,所在城市,海德公园,景点,19,伦敦,城市,16
997,明孝陵坐落在南京市紫金山南麓独龙阜玩珠峰下，东毗中山陵，南临梅花山，是南京最大的帝王陵墓，亦...,所在城市,中山陵,景点,24,南京,城市,6
998,2000年伊始，天津市政府对马场道进行大规模的整修，作为当年为民办十件实事之一,所在城市,马场道,景点,14,天津市,城市,8


In [5]:
#加载csv
def load_csv(file):
    data_list = []

    with codecs.open(filename=file,encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for line in reader:
            data = list(line.values())
            data_list.append(data)
    return data_list

In [6]:
train_raw_data = load_csv(r'train.csv')
test_raw_data = load_csv(r'test.csv')
type(train_raw_data)
train_raw_data[:5]

[['《逐风行》是百度文学旗下纵横中文网签约作家清水秋风创作的一部东方玄幻小说，小说已于2014-04-28正式发布',
  '连载网站',
  '逐风行',
  '网络小说',
  '1',
  '纵横中文网',
  '网站',
  '12'],
 ['谢万松，字树人，湖北省武汉市人，武汉钢铁集团公司联合焦化公司退体职工，生于1940年',
  '出生地',
  '谢万松',
  '人物',
  '0',
  '湖北省武汉市',
  '地点',
  '8'],
 ['《娘家的故事第二部》是张玲执导，林在培、何赛飞等主演的电视剧',
  '导演',
  '娘家的故事第二部',
  '影视作品',
  '1',
  '张玲',
  '人物',
  '11'],
 ['九玄珠是在纵横中文网连载的一部小说，作者是龙马', '连载网站', '九玄珠', '网络小说', '0', '纵横中文网', '网站', '5'],
 ['《下半生留住你一直相爱》是商人醉猫写的网络小说连载于17k小说网',
  '连载网站',
  '下半生留住你一直相爱',
  '网络小说',
  '1',
  '17k小说网',
  '网站',
  '26']]

In [104]:
#获取训练接和数据集各自的关系集合
train_relation = [] # 训练集中的所有关系
test_relation = [] # 测试集中的所有关系
for data in train_raw_data:
    train_relation.append(data[1])
for data in test_raw_data:
    test_relation.append(data[1])

In [105]:
train_relation[:10]

['连载网站', '出生地', '导演', '连载网站', '连载网站', '国籍', '国籍', '连载网站', '毕业院校', '导演']

In [106]:
test_relation[:10]

['导演', '导演', '导演', '导演', '导演', '导演', '导演', '导演', '导演', '导演']

In [107]:
train_raw_data[:1]

[['《逐风行》是百度文学旗下纵横中文网签约作家清水秋风创作的一部东方玄幻小说，小说已于2014-04-28正式发布',
  '连载网站',
  '逐风行',
  '网络小说',
  '1',
  '纵横中文网',
  '网站',
  '12',
  ['《',
   '网络小说',
   '》',
   '是',
   '百度',
   '文学',
   '旗下',
   '网站',
   '签约',
   '作家',
   '清水',
   '秋风',
   '创作',
   '的',
   '一部',
   '东方',
   '玄幻',
   '小说',
   '，',
   '小说',
   '已于',
   '2014',
   '-',
   '04',
   '-',
   '28',
   '正式',
   '发布'],
  [1, 7]]]

In [108]:
import jieba

In [109]:
#句子分词
def split_sentences(raw_data):
    new_data_list = []
    jieba.add_word('HEAD')
    jieba.add_word('TAIL')
    for data in raw_data:
        head, tail = data[3], data[6]
        new_sentence = data[0].replace(data[2], 'HEAD', 1) # 替换句子中的实体关键词
        new_sentence = new_sentence.replace(data[5], 'TAIL', 1)
        
        new_sentence = jieba.lcut(new_sentence)
        head_pos, tail_pos = new_sentence.index('HEAD'), new_sentence.index('TAIL')
        
        new_sentence[head_pos] = head 
        new_sentence[tail_pos] = tail
        data.append(new_sentence)
        data.append([head_pos, tail_pos])
        
        new_data_list.append(data)
    return new_data_list

In [110]:
train_raw_data = split_sentences(train_raw_data)
test_raw_data = split_sentences(test_raw_data)

In [111]:
train_raw_data[0]

['《逐风行》是百度文学旗下纵横中文网签约作家清水秋风创作的一部东方玄幻小说，小说已于2014-04-28正式发布',
 '连载网站',
 '逐风行',
 '网络小说',
 '1',
 '纵横中文网',
 '网站',
 '12',
 ['《',
  '网络小说',
  '》',
  '是',
  '百度',
  '文学',
  '旗下',
  '网站',
  '签约',
  '作家',
  '清水',
  '秋风',
  '创作',
  '的',
  '一部',
  '东方',
  '玄幻',
  '小说',
  '，',
  '小说',
  '已于',
  '2014',
  '-',
  '04',
  '-',
  '28',
  '正式',
  '发布'],
 [1, 7],
 ['《',
  '网络小说',
  '》',
  '是',
  '百度',
  '文学',
  '旗下',
  '网站',
  '签约',
  '作家',
  '清水',
  '秋风',
  '创作',
  '的',
  '一部',
  '东方',
  '玄幻',
  '小说',
  '，',
  '小说',
  '已于',
  '2014',
  '-',
  '04',
  '-',
  '28',
  '正式',
  '发布'],
 [1, 7]]

In [112]:
train_raw_data[0][-2]

['《',
 '网络小说',
 '》',
 '是',
 '百度',
 '文学',
 '旗下',
 '网站',
 '签约',
 '作家',
 '清水',
 '秋风',
 '创作',
 '的',
 '一部',
 '东方',
 '玄幻',
 '小说',
 '，',
 '小说',
 '已于',
 '2014',
 '-',
 '04',
 '-',
 '28',
 '正式',
 '发布']

In [114]:
from torchtext.vocab import build_vocab_from_iterator # 创建词表工具

In [115]:
def yield_tokens(raw_data):
    for data in raw_data:
        yield data[-2]

In [116]:
# 使用训练数据集构建单词表
vocab = build_vocab_from_iterator(yield_tokens(train_raw_data), specials=["<pad>", "<unk>"], min_freq=2)


In [117]:
 vocab.set_default_index(vocab['<unk>'])

In [118]:
len(vocab)

6618

In [119]:
l1 = list(vocab.get_stoi().values())

In [120]:
vocab.get_stoi()['<unk>']

1

In [121]:
#获取位置编码：
def get_pos_feature(sent_len, entity_pos, entity_len = 1, pos_limit = 50):
    left = list(range(-entity_pos, 0))
    middle = [0] * entity_len
    right = list(range(1, sent_len - entity_pos - entity_len + 1))
    pos = left + middle + right
    for i, p in enumerate(pos):
        if p > pos_limit:
            pos[i] = pos_limit
        if p < -pos_limit:
            pos[i] = -pos_limit
    
    pos = [p + pos_limit + 1 for p in pos]
    return pos

In [122]:
def build_data(raw_data, vocab):
    sents = []
    head_pos = []
    tail_pos = []
    
    for data in raw_data:
        
        
        #sent = [vocab.get_stoi()[w] for w in data[-2]]
        sent = []
        for w in data[-2]: # 迭代取出该句的所有词
            if w in vocab: # 判断该词是否在词表中
                sent.append(vocab.get_stoi()[w]) # 有，则将居中的单词转为id
            else:
                sent.append(vocab.get_stoi()['<unk>']) # 无，则使用<unk>的id代替
        #print(type(sent))
        pos = list(range(len(sent)))
        head, tail = int(data[-1][0]), int(data[-1][-1])
        head_p = get_pos_feature(len(sent), head)
        tail_p = get_pos_feature(len(sent), tail)
        sents.append(sent)
        head_pos.append(head_p)
        tail_pos.append(tail_p)
    return sents, head_pos, tail_pos

In [123]:
train_sents, train_head_pos, train_tail_pos = build_data(train_raw_data, vocab)
test_sents, test_head_pos, test_tail_pos = build_data(test_raw_data, vocab)

In [124]:
train_sents[0]

[4,
 22,
 5,
 9,
 738,
 319,
 472,
 30,
 346,
 240,
 5653,
 3654,
 75,
 3,
 59,
 483,
 884,
 52,
 2,
 52,
 729,
 219,
 62,
 966,
 62,
 444,
 199,
 391]

In [125]:
#将训练集和测试集的关系类别转为对应的id值
def relation_token(relations, file):
    relation_list = []
    relation_dict = {}
    out = []
    with codecs.open(file, encoding='utf-8') as f:
        for line in f:
            relation_list.append(line.strip()) # 获取relaiton.txt中的关系
        for i, rel in enumerate(relation_list):
            relation_dict[rel] = i # 构建关系字典
        for rel in relations:
            out.append(relation_dict[rel]) # 获取训练或数据集中的关系转为id值
    return out

In [126]:
# 构建关系数据
relation_file = r'relation.txt'
train_relation_token = relation_token(train_relation, relation_file) 
test_relation_token = relation_token(test_relation, relation_file)

In [127]:
train_relation_token[:10] #训练集中的前10关系类别

[7, 3, 2, 7, 7, 0, 0, 7, 9, 2]

In [128]:
#将
train_data = list(
    zip(train_sents, train_head_pos, train_tail_pos, train_relation_token)
)
test_data = list(
    zip(test_sents, test_head_pos, test_tail_pos, train_relation_token)
)

In [129]:
#train_data[0]

In [130]:
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [131]:
class CustomDataset(Dataset):
    def __init__(self, file):
        super(CustomDataset, self).__init__()
        self.file = file
    
    def __getitem__(self, item):
        sample = self.file[item]
        return sample
    
    def __len__(self):
        return len(self.file)

In [132]:
# 获取训练集和测试集的dataset
train_dataset = CustomDataset(train_data)
test_dataset = CustomDataset(test_data)

In [133]:
def collate_fn(batch):
    batch.sort(key = lambda data: len(data[0]), reverse=True)
    lens = [len(data[0]) for data in batch]
    max_len = max(lens) # 找到一个批次中，最长的句子
    
    sent_list = []
    head_pos_list = []
    tail_pos_list = []
    relation_list = []
    
    def _padding(x, max_len): # 对短句子做填充
        return x + [0] * (max_len - len(x))
    
    for data in  batch:
        sent, head_pos, tail_pos, relation = data
        sent_list.append(_padding(sent, max_len))
        head_pos_list.append(_padding(head_pos, max_len))
        tail_pos_list.append(_padding(tail_pos, max_len))
        relation_list.append(relation)
        
    return torch.tensor(sent_list), torch.tensor(head_pos_list), torch.tensor(tail_pos_list), torch.tensor(relation_list)

In [134]:
train_dataloader = DataLoader(
    dataset=train_dataset,
    batch_size=128,
    shuffle=True,
    drop_last=True, # 最后一个批次数据不足时，直接丢弃
    collate_fn=collate_fn
)
test_dataloader = DataLoader(
    dataset=test_dataset,
    batch_size=128,
    shuffle=True,
    drop_last=True,
    collate_fn=collate_fn
)

In [135]:
type(train_dataloader)

torch.utils.data.dataloader.DataLoader

In [136]:
class Embedding(nn.Module):

    def __init__(self, vocab_size, word_dim, pos_size, pos_dim):
        super(Embedding, self).__init__()
        self.word_embed = nn.Embedding(vocab_size, word_dim, padding_idx=0)
        self.head_pos_embed = nn.Embedding(pos_size, pos_dim, padding_idx=0)
        self.tail_pos_embed = nn.Embedding(pos_size, pos_dim, padding_idx=0)

    def forward(self, x):
        words, head_pos, tail_pos = x
        word_embed = self.word_embed(words)
        head_embed = self.head_pos_embed(head_pos)
        tail_embed = self.tail_pos_embed(tail_pos)
        feature_embed = torch.cat([word_embed, head_embed, tail_embed], dim=-1)

        return feature_embed

In [137]:
class Config(object):
    model_name = 'BruceCNN'

    """
    文本处理
    """
    data_path = 'data/origin'  # 原始数据
    out_path = 'data/out'  # 数据处理的最终结果保存
    is_chinese = True  # 是否为中文

    word_segment = True  # 是否分词

    relation_type = 10  # 关系种类数

    min_frep = 2  # 低频词处理判断频率

    """
    位置编码
    """
    pos_limit = 50  # 【-50， 50】
    pos_size = 102  # 2*pos_limit + 2


    """
    Embedding层
    """
    word_dim = 300  # 词向量维度
    pos_dim = 10  # 位置向量维度

    """
    FC层
    """
    hidden_size = 100  # 隐藏层数
    dropout = 0.5

    """
    CNN层
    """
    out_channels = 100
    kernel_size = [3, 5]

    """
    初始化种子
    """
    seed = 1

    """
    设置GPU
    """
    use_gpu = True  # 是否使用GPU
    gpu_id = 0  # 存在多个GPU时使用的GPU编号

In [138]:
config = Config()

In [139]:
vocab_size = len(vocab)

# PCNN关系抽取

In [140]:
class BruceCNN(nn.Module):
    def __init__(self, vocab_size, config):
        super(BruceCNN, self).__init__()
        self.model_name = 'BruceCNN'
        self.out_channels = config.out_channels
        self.kernel_size = config.kernel_size

        self.word_dim = config.word_dim
        self.pos_dim = config.pos_dim
        self.pos_size = config.pos_size

        self.hidden_size = config.hidden_size
        self.dropout = config.dropout
        self.out_dim = config.relation_type #fc层的输出维度为关系分类的数量

        """
        isinstance(object, classinfo) 判断一个对象是否已知的类型
        """
        if isinstance(self.kernel_size, int):
            self.kernel_size = [self.kernel_size]
        for k in self.kernel_size:
            assert k % 2 == 1

        """
        Embedding嵌入层
        """
        self.embedding = Embedding(vocab_size, self.word_dim, self.pos_size, self.pos_dim)

        self.input_dim = self.word_dim + self.pos_dim * 2  #将词向量与位置向量维度拼接后，作为卷积层的输入

        """
        卷积层
        """
        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=self.input_dim,
                      out_channels=self.out_channels,
                      kernel_size=k,
                      padding=k // 2,
                      bias=None) for k in self.kernel_size
        ])

        self.conv_dim = len(self.kernel_size) * self.out_channels #textCNN模型输出后做拼接

        """
        fc层
        """
        self.fc1 = nn.Linear(self.conv_dim, self.hidden_size)
        self.fc2 = nn.Linear(self.hidden_size, self.out_dim)

    def forward(self, input):
        """

        :param input:word_ids, headpos, tailpos, mask
        :return:
        """
        # *x, mask = input
        x = input
        
        x = self.embedding(x)
        x = torch.transpose(x, 1, 2) #将x的第一维度和第二维度互换




        x = [F.leaky_relu(conv(x)) for conv in self.convs]  # 获取两种卷积的结果
        x = torch.cat(x, dim=1)
        x = F.max_pool1d(x, x.size(-1))
        x = x.squeeze(-1)
        #print(x)
        #x = self.dropout(x)

        x = F.leaky_relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))

        return x

In [141]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

In [142]:
model = BruceCNN(vocab_size=vocab_size, config=config).to(device)

In [143]:
print(model)

BruceCNN(
  (embedding): Embedding(
    (word_embed): Embedding(6618, 300, padding_idx=0)
    (head_pos_embed): Embedding(102, 10, padding_idx=0)
    (tail_pos_embed): Embedding(102, 10, padding_idx=0)
  )
  (convs): ModuleList(
    (0): Conv1d(320, 100, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    (1): Conv1d(320, 100, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
  )
  (fc1): Linear(in_features=200, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
)


In [144]:
from torch.optim import lr_scheduler
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
scheduler = lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
loss_fn = nn.CrossEntropyLoss()

In [145]:
from sklearn.metrics import precision_recall_fscore_support

In [146]:
def train(epoch, device, data_loader, model, optimizer, loss_fn):
    model.train()
    total_loss = []
    
    for batch_idx, batch in enumerate(data_loader):
        *x, y = [data.to(device) for data in batch]
        optimizer.zero_grad()
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        loss.backward()
        optimizer.step()
        total_loss.append(loss)
        
        # 记录日志
        data_cal = len(data_loader.dataset) if batch_idx == len(data_loader) else batch_idx * len(y)
        
        if(batch_idx % 10 == 0) or batch_idx == len(data_loader):
            print('Training epoch:{} [{} / {} ({:.0f}%)] \t Loss:{:.6f}'.format(epoch, 
                                                                                 data_cal, 
                                                                                 len(data_loader.dataset), 
                                                                                 100.*batch_idx / len(data_loader), 
                                                                                 loss.item()))
    #print('************************epoch结束********************************')       

In [147]:
def validate(data_loader, model, device,):
    model.eval()
    
    with torch.no_grad():
        total_y_true = np.empty(0)
        total_y_pred = np.empty(0)
        
        for batch_idx, batch in enumerate(data_loader):
            *x, y = [data.to(device) for data in batch]
            y_pred = model(x)
            y_pred = y_pred.argmax(dim = -1)
            
            try:
                y, y_pred = y.numpy(), y_pred.numpy()
            except:
                y, y_pred = y.cpu().numpy(), y_pred.cpu().numpy()
            
            total_y_true = np.append(total_y_true, y)
            total_y_pred = np.append(total_y_pred, y_pred)
         
        total_f1 = []
        
        for average in ['macro', 'micro']:
            precision, recall, f1, _ = precision_recall_fscore_support(total_y_true, total_y_pred, average=average)
            
            print(f'{average} metrics: [precision: {precision: .4f}, recall: {recall: .4f}, f1: {f1: .4f}]')
            
            total_f1.append(f1)
        print('*************************epoch结束************************************')    
    return total_f1

In [148]:
for epoch in range(1, 11):
    train(epoch, device, train_dataloader, model, optimizer, loss_fn)
    macro_f1, micro_f1 = validate(test_dataloader, model, device)

Training epoch:1 [0 / 4000 (0%)] 	 Loss:2.344216
Training epoch:1 [1280 / 4000 (32%)] 	 Loss:24.179258
Training epoch:1 [2560 / 4000 (65%)] 	 Loss:20.385035
Training epoch:1 [3840 / 4000 (97%)] 	 Loss:44.534355
macro metrics: [precision:  0.0395, recall:  0.0979, f1:  0.0284]
micro metrics: [precision:  0.1417, recall:  0.1417, f1:  0.1417]
*************************epoch结束************************************
Training epoch:2 [0 / 4000 (0%)] 	 Loss:21.287914


D:\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training epoch:2 [1280 / 4000 (32%)] 	 Loss:4.668078
Training epoch:2 [2560 / 4000 (65%)] 	 Loss:13.757561
Training epoch:2 [3840 / 4000 (97%)] 	 Loss:2.308952


D:\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


macro metrics: [precision:  0.0404, recall:  0.1042, f1:  0.0108]
micro metrics: [precision:  0.0234, recall:  0.0234, f1:  0.0234]
*************************epoch结束************************************
Training epoch:3 [0 / 4000 (0%)] 	 Loss:2.279138
Training epoch:3 [1280 / 4000 (32%)] 	 Loss:2.302521
Training epoch:3 [2560 / 4000 (65%)] 	 Loss:2.769456
Training epoch:3 [3840 / 4000 (97%)] 	 Loss:2.334203
macro metrics: [precision:  0.0616, recall:  0.1051, f1:  0.0117]
micro metrics: [precision:  0.0212, recall:  0.0212, f1:  0.0212]
*************************epoch结束************************************
Training epoch:4 [0 / 4000 (0%)] 	 Loss:2.332601
Training epoch:4 [1280 / 4000 (32%)] 	 Loss:2.258194
Training epoch:4 [2560 / 4000 (65%)] 	 Loss:1.990674


...

D:\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training epoch:9 [1280 / 4000 (32%)] 	 Loss:3.039879
Training epoch:9 [2560 / 4000 (65%)] 	 Loss:7.719632
Training epoch:9 [3840 / 4000 (97%)] 	 Loss:2.380845


D:\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


macro metrics: [precision:  0.0562, recall:  0.0967, f1:  0.0096]
micro metrics: [precision:  0.0179, recall:  0.0179, f1:  0.0179]
*************************epoch结束************************************
Training epoch:10 [0 / 4000 (0%)] 	 Loss:2.045395
Training epoch:10 [1280 / 4000 (32%)] 	 Loss:10.711118
Training epoch:10 [2560 / 4000 (65%)] 	 Loss:2.047909
Training epoch:10 [3840 / 4000 (97%)] 	 Loss:2.115601
macro metrics: [precision:  0.0860, recall:  0.1009, f1:  0.0108]
micro metrics: [precision:  0.0190, recall:  0.0190, f1:  0.0190]
*************************epoch结束************************************


D:\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 用GRU的关系抽取

In [183]:
#
class GRU(nn.Module):
    def __init__(self, vocab_size, config):
        super(GRU, self).__init__()
        self.model_name = 'GRU'
        self.word_dim = config.word_dim
        self.pos_size = config.pos_size
        self.pos_dim = config.pos_dim
        self.lstm_layers = 2
        self.hidden_size = config.hidden_size
        self.dropout = config.dropout
        self.out_dim= config.relation_type
        
        self.embedding = Embedding(vocab_size, self.word_dim, self.pos_size, self.pos_dim)
        
        self.input_dim = self.word_dim + self.pos_dim * 2
        
        self.gru = nn.GRU(
            input_size = self.input_dim,
            hidden_size = self.hidden_size,
            num_layers = self.lstm_layers,
            dropout = self.dropout,
            bidirectional = True,
            bias = True,
            batch_first = True
        )
        
        linear_input_dim = self.hidden_size
        self.fc1 = nn.Linear(linear_input_dim, self.hidden_size)
        self.fc2 = nn.Linear(self.hidden_size, self.out_dim)
    
    def forward(self, input):
        x= input
        x = self.embedding(x)
        out_put, h_n = self.gru(x)
        h_n = h_n[-1,:,:]
        y = F.leaky_relu(self.fc1(h_n))
        out = F.leaky_relu(self.fc2(y))
        return out

In [184]:
gru_model = GRU(vocab_size, config).to(device)

In [185]:
gru_model

GRU(
  (embedding): Embedding(
    (word_embed): Embedding(6618, 300, padding_idx=0)
    (head_pos_embed): Embedding(102, 10, padding_idx=0)
    (tail_pos_embed): Embedding(102, 10, padding_idx=0)
  )
  (gru): GRU(320, 100, num_layers=2, batch_first=True, dropout=0.5, bidirectional=True)
  (fc1): Linear(in_features=100, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
)

In [171]:
for epoch in range(1, 11):
    train(epoch, device, train_dataloader, bilstm_model, optimizer, loss_fn)
    macro_f1, micro_f1 = validate(test_dataloader, bilstm_model, device)

Training epoch:1 [0 / 4000 (0%)] 	 Loss:2.300079
Training epoch:1 [1280 / 4000 (32%)] 	 Loss:2.294780
Training epoch:1 [2560 / 4000 (65%)] 	 Loss:2.302145
Training epoch:1 [3840 / 4000 (97%)] 	 Loss:2.300515
macro metrics: [precision:  0.0323, recall:  0.0988, f1:  0.0438]
micro metrics: [precision:  0.1094, recall:  0.1094, f1:  0.1094]
*************************epoch结束************************************
Training epoch:2 [0 / 4000 (0%)] 	 Loss:2.298873


D:\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training epoch:2 [1280 / 4000 (32%)] 	 Loss:2.299347
Training epoch:2 [2560 / 4000 (65%)] 	 Loss:2.302216
Training epoch:2 [3840 / 4000 (97%)] 	 Loss:2.296124
macro metrics: [precision:  0.0394, recall:  0.1001, f1:  0.0485]
micro metrics: [precision:  0.1194, recall:  0.1194, f1:  0.1194]
*************************epoch结束************************************
Training epoch:3 [0 / 4000 (0%)] 	 Loss:2.292578


D:\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training epoch:3 [1280 / 4000 (32%)] 	 Loss:2.295928
Training epoch:3 [2560 / 4000 (65%)] 	 Loss:2.301129
Training epoch:3 [3840 / 4000 (97%)] 	 Loss:2.299601
macro metrics: [precision:  0.0327, recall:  0.0997, f1:  0.0455]
micro metrics: [precision:  0.1150, recall:  0.1150, f1:  0.1150]
*************************epoch结束************************************
Training epoch:4 [0 / 4000 (0%)] 	 Loss:2.302959


...

D:\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training epoch:9 [1280 / 4000 (32%)] 	 Loss:2.310285
Training epoch:9 [2560 / 4000 (65%)] 	 Loss:2.299006
Training epoch:9 [3840 / 4000 (97%)] 	 Loss:2.305078
macro metrics: [precision:  0.0319, recall:  0.0958, f1:  0.0443]
micro metrics: [precision:  0.1116, recall:  0.1116, f1:  0.1116]
*************************epoch结束************************************
Training epoch:10 [0 / 4000 (0%)] 	 Loss:2.296948


D:\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Training epoch:10 [1280 / 4000 (32%)] 	 Loss:2.299944
Training epoch:10 [2560 / 4000 (65%)] 	 Loss:2.301326
Training epoch:10 [3840 / 4000 (97%)] 	 Loss:2.300161
macro metrics: [precision:  0.0380, recall:  0.1007, f1:  0.0473]
micro metrics: [precision:  0.1150, recall:  0.1150, f1:  0.1150]
*************************epoch结束************************************


D:\miniconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# GRU+注意力机制

In [153]:
#定义注意力机制
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        
        self.method = method
        self.hidden_size = hidden_size
        
        
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
            
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(1, hidden_size))
    
    def forward(self, hidden, encoder_outputs):
        max_len = encoder_outputs.size(0)
        batch_size = encoder_outputs.size(1)
            
        attn_energies = torch.zeros(batch_size, max_len)
        attn_energies = attn_energies.to(device)
        
        for n in range(batch_size):
            for i in range(max_len):
                attn_energies[b, i] = self.score(hidden[: b], encoder_outputs[i, b].unsqueeze(0))
        return F.softmax(attn_energies, dim=1).unsqueeze(1)
    
    # 定义注意力打分函数
    def score(self, hidden, encoder_output):
        if self.method == 'dot':
            energy = hidden.squeeze(0).dot(encoder_output.squeeze(0))
            return energy
        
        elif self.method == 'general':
            energy = self.attn(encoder_output)
            energy = hidden.squeeze(0).dot(energy.squeeze(0))
            return energy
        
        elif self.method == 'concat':
            energy = self.attn(torch.cat(hidden, encoder_output), 1)
            energy = self.v.squeeze(0).dot(energy.squeeze(0))
            return energy

In [172]:
class AttnGRU(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers = 1, dropout = 0.1):
        super(AttnGRU, self).__init__()
        
        # 定义参数
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout
        
        # 定义层
        self.embedding = embedding
        self.embedding_drop = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout = (0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        
        # 选择注意力机制
        if attn_model != 'none':
            self.attn = Attn(attn_model, hidden_size)
    
    def forward(self, input_seq, last_hidden, encoder_outputs):
        embedded = self.embedding(input_seq)
        embedded = self.embedding_drop(embedded)
        # if(embedded.size(0) != 1):
        #    raiseValueError('Decoder input sequence length should be 1')
        rnn_output, hidden = self.gru(embedded, last_hidden) # 经过bilstm层
        
        attn_weights = self.attn(rnn_output, encoder_outputs)
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        
        # 注意力向量使用隐藏层状态和context向量
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tan(self.concat(concat_input))
        output = self.out(concat_output)
        
        return output, hidden, attn_weights

In [189]:
attn_model = Attn('dot', 100)
attn_model

Attn()

In [186]:
embedding = Embedding(vocab_size, config.word_dim, config.pos_size, config.pos_dim)
embedding

Embedding(
  (word_embed): Embedding(6618, 300, padding_idx=0)
  (head_pos_embed): Embedding(102, 10, padding_idx=0)
  (tail_pos_embed): Embedding(102, 10, padding_idx=0)
)

In [187]:
attnGru = AttnGRU(attn_model, embedding, config.hidden_size, config.relation_type)
attnGru

AttnGRU(
  (attn_model): Attn()
  (embedding): Embedding(
    (word_embed): Embedding(6618, 300, padding_idx=0)
    (head_pos_embed): Embedding(102, 10, padding_idx=0)
    (tail_pos_embed): Embedding(102, 10, padding_idx=0)
  )
  (embedding_drop): Dropout(p=0.1, inplace=False)
  (gru): GRU(100, 100)
  (concat): Linear(in_features=200, out_features=100, bias=True)
  (out): Linear(in_features=100, out_features=10, bias=True)
  (attn): Attn(
    (method): Attn()
  )
)

In [182]:
for epoch in range(1, 11):
    train(epoch, device, train_dataloader, attnGru, optimizer, loss_fn)
    macro_f1, micro_f1 = validate(test_dataloader, attnGru, device)

TypeError: forward() missing 2 required positional arguments: 'last_hidden' and 'encoder_outputs'